<a href="https://colab.research.google.com/github/agi2019/ppi-gci/blob/main/tutorials/01c%20-%20data%20preparation%20(expenditure).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Data preparation – expenditure</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>) Adapted for PPI-Global Cybersecurity Index (GCI) project on cybersecurity policy prioritisation

This tutorial will show you how to prepare a dataset containing expenditure programmes that are linked, somehow, to the cybersecurity indicators. The raw expenditure data has already structured and pre-processed. The aim is to prepare two files:
1.	A disbursement schedule.
2.	A relational table.

## Import the necessary Python libraries to manipulate data

In [ ]:
import pandas as pd
import numpy as np

## On expenditure linked data

In any impact evaluation of public expenditure, it is necessary to obtain information about the level of expenditure that is dedicated to government programmes aiming at improving an indicator. Usually, broad tranches such as education, public health, or national defence are used for that purpose. In the context of multidimensional impact evaluation, these data need to be more disaggregated. In an ideal scenario, there should be one development indicator directly linked to one expenditure programme. In real life, such mapping is more complex because there can be multiple government programmes designed to affect the same indicator, or several indicators affected by the same programme. PPI was designed with these structures in mind.

Today, it is still difficult to find expenditure datasets with a high degree of disaggregation. Thus, in this tutorial, I will show an example where the SDGs serve the function of linking categories. Before elaborating in this example, I need to explain how PPI uses the expenditure data and some important considerations that one needs to take into account before preparing the final dataset.


## PPI and expenditure data

To accommodate different linkages between expenditure and indicators with varied quality, PPI relies on a model of how the government prioritises its spending (see more in the Model chapter of the book). For instance, if there is only aggregate data for the tranche of education, and there are several indicators capturing different policy issues within education, the model determines the spending distribution within this tranche endogenously. However, if the user has data on how the educational budget was actually allocated across more fine-grained policy issues, this information can be incorporated into PPI to rely less on the model and more on the data.


## Temporal factors

Before preparing the example datasets, it is important to mention three important adjustments that should be done beforehand:

* Accounting for inflation.
* Accounting for population growth.
* Accounting for spending inertia.

Controlling for these (and perhaps other) temporal variables is important to remove their influence from the expenditure-indicator relationship that PPI models. Dealing for inflation is straightforward as it consists of turning the expenditure time series in constant monetary units. Removing the influence of population growth is also easy to do as one needs to divide the data by the population size (which changes through time) to obtain per capita spending.

Once the two previous adjustments have been made, there may still be certain inertia in the spending time series. This should also be removed for technical reasons related to the model in PPI. In a nutshell, this is necessary because of the calibration of a parameter $\beta_i$ that normalises the expenditure on indicator $i$ in the 0-1 range to determine the probability of success of the indicator.

Note that $\beta_i$ does not have a time sub-index, so it is a constant parameter. This means that, if the expenditure data related to $i$ has a positive trend, the latter periods of the simulation will tend to have a higher probability of success than the early ones. These inter-temporal differences in success rates is an artefact of not removing the trend component of expenditure, as the indicator data do not suggest a systematic improvement in success rates, but rather that spending in policy issues becomes more expensive–in real terms–with time.

There are various ways in which one could remove the trend component from an expenditure time series. In the book, we use the naive approach of simply calculating the inter-temporal average of each expenditure programme, and applying it in every period. This is a simple approach that meets the technical level of the book, and that is acceptable if one is not concerned about specific points in time during the sample period. However, one may want to consider more nuanced methods like running a linear regression and taking the differences with respect to the predicted values, or a Hodrick–Prescott filter that is popular in macroeconomics, or other signal-processing methods stemming from engineering (as I do in http://dx.doi.org/10.2139/ssrn.4101378).

In this tutorial, I assume that the raw expenditure data consist of a table with time series on government spending on various SDGs; and that these data have already been prepared to account for the aforementioned factors. Hence, in absence of expenditure programmes, the SDGs provide the imperfect link between spending data and indicators.

## Load expenditure data

In [ ]:
data_exp = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/raw_data/raw_expenditure.csv')
data_indi = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_indicators.csv')

In [ ]:
data_exp

,sdg,2020,2021,2022,2023,2024
0,11,6.936540e+08,2.207260e+08,604656000,3864677000,1.420850e+09
1,12,3.603463e+09,2.316681e+09,4256757000,7270508000,2.930907e+09
2,21,4.047520e+11,6.192900e+11,53903749000,4860150000,1.823350e+10
3,22,3.278602e+09,1.142292e+10,11705945000,9219936000,7.791983e+09
4,23,2.205917e+09,1.800895e+09,631924000,1331138000,1.650166e+09
5,31,0.000000e+00,0.000000e+00,537171000,820967000,1.165564e+09
6,32,4.454448e+10,9.849515e+10,29873249000,12693443000,4.060620e+11
7,33,6.831659e+09,7.437798e+09,5128202000,9432893000,4.590403e+09
8,34,0.000000e+00,0.000000e+00,0,0,0.000000e+00
9,41,6.373120e+09,8.681508e+09,6038437000,4711456000,2.854410e+09


First, let me check which SDGs are covered in both the expenditure and the indicators datasets. Then, I need to make sure that every SDG has at least one instrumental indicator.

In [ ]:
data_exp = data_exp[data_exp.sdg.isin(data_indi.sdg.values)]
data_exp = data_exp[data_exp.sdg.isin(data_indi[data_indi.instrumental==1].sdg.values)]

In [ ]:
data_exp

,sdg,2020,2021,2022,2023,2024
0,11,6.936540e+08,2.207260e+08,604656000,3864677000,1.420850e+09
1,12,3.603463e+09,2.316681e+09,4256757000,7270508000,2.930907e+09
2,21,4.047520e+11,6.192900e+11,53903749000,4860150000,1.823350e+10
3,22,3.278602e+09,1.142292e+10,11705945000,9219936000,7.791983e+09
4,23,2.205917e+09,1.800895e+09,631924000,1331138000,1.650166e+09
5,31,0.000000e+00,0.000000e+00,537171000,820967000,1.165564e+09
6,32,4.454448e+10,9.849515e+10,29873249000,12693443000,4.060620e+11
7,33,6.831659e+09,7.437798e+09,5128202000,9432893000,4.590403e+09
8,34,0.000000e+00,0.000000e+00,0,0,0.000000e+00
9,41,6.373120e+09,8.681508e+09,6038437000,4711456000,2.854410e+09


## Assembling disbursement schedule

In PPI, each simulation runs for $T$ steps. Each of these simulation steps <strong>does not</strong> correspond to a calendar period. Thus, the user needs to convert the expenditure data into a format that matches the number of simulation steps. Since most indicator and expenditure data are relatively short, $T$ tends to be larger than the sample period. Therefore, the expenditure series need to be "streched" to provide the government agent with a disbursement schedule that determines the level of spending in each simulation step.

For these tutorials, let me use $T=60$, so that three simulation steps are equivalent to one calendar year (because there are5 years in the data). The Calibration chapter of the book explains that the choice of $T$ should be at least 40 to give enough time to the agents to learn and establish a social norm of inefficiency. Values beyond 50 do not change the results in a significant way, but rather impose a computational burden. Thus, it is recommended to establish a $T$ between 40 and 200 that is easy to translate into calendar time.

$T=60$ means that the government spends every 1 months. To adjust the data to $T$, I will assume that, within each year, the government spends the same amount every 1 months. Effectively, this means that one only needs to divide each annual observation by 12 and repeat it twelve times to elongate the time series. Of course, one does not need to assume an even temporal split. If nuanced information about how the government spends its resources is available, it is easy to introduce a more detailed disbursement schedule.

In [ ]:
years = [column_name for column_name in data_exp.columns if str(column_name).isnumeric()]
periods = len(years)
T = 60
t = int(T/periods)

new_rows = []
for index, row in data_exp.iterrows():
    new_row = [row.sdg]
    for year in years:
        new_row += [int(row[year]) for i in range(t)]
    new_rows.append(new_row)

df_exp = pd.DataFrame(new_rows, columns=['sdg']+list(range(T)))

In [ ]:
df_exp

,sdg,0,1,2,3,4,5,6,7,8,...,50,51,52,53,54,55,56,57,58,59
0,11.0,693654000,693654000,693654000,693654000,693654000,693654000,693654000,693654000,693654000,...,1420850000,1420850000,1420850000,1420850000,1420850000,1420850000,1420850000,1420850000,1420850000,1420850000
1,12.0,3603463000,3603463000,3603463000,3603463000,3603463000,3603463000,3603463000,3603463000,3603463000,...,2930907000,2930907000,2930907000,2930907000,2930907000,2930907000,2930907000,2930907000,2930907000,2930907000
2,21.0,404752000000,404752000000,404752000000,404752000000,404752000000,404752000000,404752000000,404752000000,404752000000,...,18233502000,18233502000,18233502000,18233502000,18233502000,18233502000,18233502000,18233502000,18233502000,18233502000
3,22.0,3278602000,3278602000,3278602000,3278602000,3278602000,3278602000,3278602000,3278602000,3278602000,...,7791983000,7791983000,7791983000,7791983000,7791983000,7791983000,7791983000,7791983000,7791983000,7791983000
4,23.0,2205917000,2205917000,2205917000,2205917000,2205917000,2205917000,2205917000,2205917000,2205917000,...,1650166000,1650166000,1650166000,1650166000,1650166000,1650166000,1650166000,1650166000,1650166000,1650166000
5,31.0,0,0,0,0,0,0,0,0,0,...,1165564000,1165564000,1165564000,1165564000,1165564000,1165564000,1165564000,1165564000,1165564000,1165564000
6,32.0,44544479000,44544479000,44544479000,44544479000,44544479000,44544479000,44544479000,44544479000,44544479000,...,406062000000,406062000000,406062000000,406062000000,406062000000,406062000000,406062000000,406062000000,406062000000,406062000000
7,33.0,6831659000,6831659000,6831659000,6831659000,6831659000,6831659000,6831659000,6831659000,6831659000,...,4590403000,4590403000,4590403000,4590403000,4590403000,4590403000,4590403000,4590403000,4590403000,4590403000
8,34.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,41.0,6373120000,6373120000,6373120000,6373120000,6373120000,6373120000,6373120000,6373120000,6373120000,...,2854410000,2854410000,2854410000,2854410000,2854410000,2854410000,2854410000,2854410000,2854410000,2854410000


## The relational table

The last part of this tutorial consists of building a dataset linking each expenditure programme to the indicators that it is supposed to affect. This dataset consists of a table with the unique ids of the indicators in the first column, and the ids of the programmes that are designed to affect them along the rest of the columns (as many columns as needed). Recall that, in this tutorial the SDGs act as programmes. In addition, you need to make sure that:

* All instrumental indicators are reported in this table.
* All programmes (GCIs) that appear in the disbursement schedule also appear in this table.
* All the indicators reported in this table are only instrumental.
* All programmes (GCIs) are designed to affect at least one instrumental indicator.

Failure to comply with any of these requirements will make PPI to throw an exception.

In [ ]:
is_instrumental = dict(zip(data_indi.seriesCode, data_indi.instrumental==1))

rel_dict = dict([(code, []) for code in data_indi.seriesCode if is_instrumental[code]])
for index, row in data_indi.iterrows():
    if row.seriesCode in rel_dict:
        rel_dict[row.seriesCode].append(row.sdg)

n_cols = max([len(value) for value in rel_dict.values()])

M = [['' for i in range(n_cols+1)] for code in rel_dict.values()]
for i, items in enumerate(rel_dict.items()):
    sdg, indis = items
    M[i][0] = sdg
    for j, indi in enumerate(indis):
        M[i][j+1] = indi

df_rel = pd.DataFrame(M, columns=['seriesCode']+list(range(n_cols)))

In [ ]:
df_rel

,seriesCode,0
0,gci11_Lonline,11
1,gci11_Lforgery,11
2,gci11_Lolsafety,11
3,gci12_Rpdp,12
4,gci12_Rprivacy,12
...,...,...
58,gci52_Mcapacitydev,52
59,gci53_Mlat,53
60,gci54_PPPdomestic,54
61,gci54_PPPforeign,54


## Save data

In [ ]:
df_exp.to_csv('data_expenditure.csv', index=False)


In [ ]:
df_rel.to_csv('data_relational_table.csv', index=False)